<a href="https://colab.research.google.com/github/calisolo/Levels_image_captioning_NICE/blob/master/Levels_train_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content

!git clone https://github.com/calisolo/Levels_image_captioning_NICE.git
!git clone https://huggingface.co/datasets/calisolo/NICE_tsv/  #dataset

%cd ./Levels_image_captioning_NICE

/content
Cloning into 'Levels_image_captioning_NICE'...
remote: Enumerating objects: 698, done.
remote: Counting objects: 100% (168/168), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 698 (delta 80), reused 121 (delta 46), pack-reused 530
Receiving objects: 100% (698/698), 89.39 MiB | 16.52 MiB/s, done.
Resolving deltas: 100% (311/311), done.
Cloning into 'NICE_tsv'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), 1.25 KiB | 642.00 KiB/s, done.
Filtering content: 100% (2/2), 890.00 MiB | 20.03 MiB/s, done.
/content/Levels_image_captioning_NICE


In [2]:
!pip install transformers==4.20.0
!pip install loguru
!pip install pycocoevalcap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 112.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 7.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 15.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

##make tokenizer!

In [ ]:
#카이밍초기화 필요? torch.nn.init.kaiming_uniform_(model.score.weight)

intent_list = []
meaningful_tags = ['[cosHint lv4]','[cosHint lv3]','[cosHint lv2]','[cosHint lv1]', '[diffHint lv3]','[diffHint lv2]','[diffHint lv1]','[shot_style]','[Location]','[NULL]']
cat_tags = ['[outdoors]', '[misc]', '[office]', '[electronics]', '[food]', '[indoors]','[medical]', '[stocky_setting]', '[animals]']
#intent_list += real_add_word
intent_list += meaningful_tags
intent_list += cat_tags

In [ ]:
intent_list

['[cosHint lv4]',
 '[cosHint lv3]',
 '[cosHint lv2]',
 '[cosHint lv1]',
 '[diffHint lv3]',
 '[diffHint lv2]',
 '[diffHint lv1]',
 '[shot_style]',
 '[Location]',
 '[NULL]',
 '[outdoors]',
 '[misc]',
 '[office]',
 '[electronics]',
 '[food]',
 '[indoors]',
 '[medical]',
 '[stocky_setting]',
 '[animals]']

In [ ]:
from component.ofa.tokenization_ofa import OFATokenizer
tokenizer = OFATokenizer.from_pretrained('OFA-Sys/ofa-huge' ,additional_special_tokens = intent_list)
tokenizer.save_pretrained('./vocab')

OFA-Sys/ofa-huge
<super: <class 'OFATokenizer'>, <OFATokenizer object>>


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


('./vocab/tokenizer_config.json',
 './vocab/special_tokens_map.json',
 './vocab/vocab.json',
 './vocab/merges.txt',
 './vocab/added_tokens.json')

## Train

In [4]:
#Train
#freeze encoder 검증
#freeze ->unfreeze?  unfreeze ->freeze?
# freeze word embed, warmupstep,max_len, caption에 제일좋은 가중치
#  scst? , 대소문자, fp 32, 길이비교( 다소 클수있음),tag?
# cosine 어닐링 구현형태 검증

#제출본에서의 역공학
#{'testlen': 235366, 'reflen': 238872, 'guess': [235366, 213989, 192613, 171240], 'correct': [129549, 78068, 52116, 36133]} BLEU에 대한 평가내역.. 딱히 이걸로 유추할수있는 정보가있나?.. 단어생성스타일은 매우 유사해졌다는정도.. 
#BLEU/Cider metric의 관계확인 #BLEU 고득점 = 말투는 비슷한데 다른 정답에서 쓴 단어를 찾아낸경우 #추가제출할때 내 제출본과 BLEU검증가능할듯
#ratio: 0.9853226832780695
# 토큰 re처리 # 안해도될듯
# WARNING: Untokenizable: � (U+FFFD, decimal: 65533)
# PTBTokenizer tokenized 260248 tokens at 935525.16 tokens per second. #정답
# Apr 23, 2023 10:49:44 AM edu.stanford.nlp.process.PTBLexer next
# WARNING: Untokenizable: � (U+FFFD, decimal: 65533)
# PTBTokenizer tokenized 256742 tokens at 930426.72 tokens per second. #4/23제출본

#30epochs = 12510 steps  -> 20epochs = 8340 steps  decay  


#import re
#result = re.sub(r'[^\w\s]', '', result).strip()
#num_beams=5, no_repeat_ngram_size=3, use_cache=False 
!CUDA_VISIBLE_DEVICES=0 python train.py --train_args_file train_args/train_ofa.json 

2023-05-27 07:27:09.184200: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-27 07:27:09.239571: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-27 07:27:10.204973: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
./vocab
<super: <class 'OFATokenizer'>, <OFATokenizer object>>
Downloading: 100% 1.36k/1.36k [00:00<00:00, 1.25MB/s]
Downloading: 100% 4.09G/4.09G [11:16<00:00, 6.50MB/s]
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: t

## Inference


In [ ]:
val_set = pd.read_csv(f'data/nice-val-5k.csv')
inference_model = 'calisolo/OFA_huge_NICE_captioning'
cosine = pd.read_csv('./data/encoder/test_fully_parsed_final_2.csv')

In [ ]:
val_dict = {}
for index, i in enumerate(val_set['public_id']):
  val_dict[i] = val_set['caption_gt'][index]

In [ ]:
#make levels hint

import ast
caption_result = {}

for index, public_id in enumerate(cosine['0']):
  processed_data = ''


  processed_data = cosine['cos1'][index] + cosine['diff1'][index] + cosine['cat1'][index]+ val_dict[cosine['key1'][index]]
  processed_data += cosine['cos2'][index] + cosine['diff2'][index] + cosine['cat2'][index]+val_dict[cosine['key2'][index]]
  processed_data += cosine['cos3'][index] + cosine['diff3'][index] + cosine['cat3'][index]+val_dict[cosine['key3'][index]]
  processed_data += cosine['cos4'][index] + cosine['diff4'][index] + cosine['cat4'][index]+val_dict[cosine['key4'][index]]
  processed_data += '[shot_style]'

  
  shotlist = ast.literal_eval(cosine['shots'][index])
  locationlist = ast.literal_eval(cosine['locations'][index])
  if shotlist == []:
    processed_data += '[NULL]'
  else:
    for shot in shotlist:
      processed_data += shot + ' | '
  processed_data += '[Location]'
  if locationlist == []:
    processed_data += '[NULL]'
  else:
    for location in locationlist:
      processed_data += location + ' | '
  caption_result[str(public_id)] = processed_data
  

In [ ]:
#inference
from component.ofa.modeling_ofa import OFAModelForCaption, OFAModel
from component.ofa.tokenization_ofa import OFATokenizer 
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import torch
import os
import base64
from datetime import date, datetime
from io import BytesIO
import datetime
test = True

csv_path = '../NICE_tsv/'

if test:
  image_file = f'{csv_path}test_image(need to delete backslash).tsv'  ## tsv(byte) read pictures faster than jpg 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = OFAModel.from_pretrained(inference_model).to(device)
print(inference_model)
tokenizer = OFATokenizer.from_pretrained('./vocab' )
model.eval()


mean, std = [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]
resolution = 256
patch_resize_transform = transforms.Compose([
        lambda image: image.convert("RGB"),
        transforms.Resize((resolution, resolution), interpolation=Image.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])

resultDF = pd.DataFrame(columns = ['public_id', 'caption'])

iter = 0
batch_size = 1
pic_list = []
save = 0
with open(image_file, 'r', encoding='utf8') as f:
  lines = f.readlines()
  with torch.no_grad():
    for line in tqdm(lines):
      pic_id, image_content = line.split('\t')
      
          
      if test:
        pic_id = pic_id[:-1]##didn't erase \\ at tsv

        txt = " what does the image describe?"
        txt+= caption_result[pic_id]
        
        inputs = tokenizer([txt], return_tensors="pt").input_ids.to(device)
        img = Image.open(BytesIO(base64.urlsafe_b64decode(image_content)))
        
      else:
        img = Image.open(f'{csv_path}val/{pic_id}.jpg')

      if iter == 0:
        patch_img = patch_resize_transform(img).unsqueeze(0).to(device)
        pic_list = [pic_id]
      else:
        imsi_img = patch_resize_transform(img).unsqueeze(0).to(device)
        patch_img = torch.cat((patch_img, imsi_img), dim = 0)
        pic_list.append(pic_id)
    
      iter+=1

      if iter== batch_size:## last batch size issue
        iter = 0
        gen = model.generate(inputs.repeat(batch_size,1), patch_images=patch_img, 
                            num_beams=5, no_repeat_ngram_size=3, max_length = 40
                            , use_cache=False )
        gen_sentence = tokenizer.batch_decode(gen, skip_special_tokens=True)
        
        ## erase if you want to full inference
        img.show()
        print(txt)
        print(gen_sentence)


        batch_df = pd.DataFrame(columns = ['public_id', 'caption'])
        for id,caption in zip(pic_list, gen_sentence):

          imsiSeries = pd.Series({'public_id':id, 'caption': caption})
          imsi_df = imsiSeries.to_frame().transpose()
          batch_df= pd.concat([batch_df,imsi_df],ignore_index = True)
        resultDF = pd.concat([resultDF,batch_df],ignore_index = True)

      # if save %2000 == 0:  save while inferencing
      #   resultDF.to_csv(f'../submission.csv', encoding = 'utf-8', index = False)
      # save+=1

resultDF.to_csv(f'../submission_final.csv', encoding = 'utf-8', index = False)

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


../drive/MyDrive/NICE/model/0501_lastcoin/checkpoint-final
./vocab
<super: <class 'OFATokenizer'>, <OFATokenizer object>>


100%|██████████| 21377/21377 [3:24:20<00:00,  1.74it/s]


In [ ]:
for i in resultDF['caption']:
  print(i)

Smiling young women sitting by fountain with one playing guitar
Vertical shot of a male and female rock climber standing on a rock
Couple on winter vacation sitting on sled on mountain top smiling at camera
View of a woman standing on a beach looking out to sea
Rooftops of idyllic village Bourdeilles Dordogne France
View of young boy concentrating on writing
Portrait of a young woman
Little girl standing with her mother and brother at the beach on an overcast windy day
Senior man sitting on a deck chair
Medium shot of the Backside of a group of friends in swimwear walking along the beach holding surfboards
Vertical shot of a businesswoman with luggage on a road in the desert
Young man pitching a baseball
A businesswoman sitting at a table thinking
Low angle view of three young women walking with shopping bags
Two young women leaning on car
Lumber industry at Breves Channels Brazil
A medium shot of a businessman and businesswoman smiling at camera while working together in front of a co

In [ ]:
for i in resultDF['caption']:
  print(i)

Smiling young women sitting by fountain with one playing guitar
Vertical shot of a male and female rock climber standing on a rock
Couple on winter vacation sitting on sled on mountain top smiling at camera
View of a woman looking out to sea
Rooftops of idyllic village Bourdeilles Dordogne France
View of young boy concentrating on writing
Portrait of a young woman
Little girl standing with her mother and brother at the beach on an overcast windy day
Senior man sitting on a deck chair
Medium shot of the Backside of a group of friends in swimwear walking along the beach holding surfboards
Vertical shot of a businesswoman with luggage on a road in the desert
Young man pitching a baseball
A businesswoman sitting at a table thinking
Low angle view of three young women walking with shopping bags
Two young women leaning on car
Lumber industry at Breves Channels Brazil
A medium shot of a businessman and businesswoman smiling at camera while working together at a desk in front of a computer
A m

##metric Test

In [ ]:
csv_path = '../drive/MyDrive/NICE/'
# val_set = pd.read_csv(f'{csv_path}/nice-val-5k.csv')
# result_path= './drive/MyDrive/NICE/output'
# resultDF = pd.read_csv(f'{result_path}/train_on_0406.csv')

val_set = pd.read_csv(f'{csv_path}/output/test_utf-8.csv', encoding = 'utf-8')
resultDF = pd.read_csv(f'{csv_path}/output/test_utf-8-sig.csv', encoding = 'utf-8')

In [ ]:
for index,i in enumerate(resultDF['caption']):
  resultDF['caption'][index] = '\ufeff' +resultDF['caption'][index] 

In [ ]:
for i in range(2500):# cider 메트릭 세부테스트
  resultDF['caption'][i] = 'A medium shot of a happy family preparing food'

In [ ]:


from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.spice.spice import Spice


combined_df = resultDF.merge(val_set, on="public_id")

dict_gt = {}
dict_pred = {}

for idx, row in combined_df.iterrows():
  dict_gt[str(row["public_id"])] = [row["caption_x"]]
  dict_pred[str(row["public_id"])] = [row["caption_y"]]
  

score, _ = Bleu(4).compute_score(dict_gt, dict_pred)
idx = 1
for s in score:
  print(f'Bleu_{idx} : {s*100}')
  idx += 1

score, _ = Rouge().compute_score(dict_gt, dict_pred)
print(f'Rouge : {score*100}')

score, _ = Cider().compute_score(dict_gt, dict_pred)
print(f'Cider : {score*100}')

score, _ = Meteor().compute_score(dict_gt, dict_pred)
print(f'Meteor : {score*100}')

score, _ = Spice().compute_score(dict_gt, dict_pred)
print(f'Spice : {score*100}')

#base(chinese -> train with chinese prompt and inferencing with English prompt) Cider  24.26
#base(chinese -> train with english prompt and inferencing with English prompt) Cider  37.21  PLM with chinese/tokenizer with Chinese
#tiny english -> english Cider 34.44  ... maybe tokenizer problem exist
#base english -> english Cider 96.46
#huge english -> english Cider 120.06 -> 98.2555!
#20 epoch -> 50 epoch  max_len 100 -> 150  Cider  399.54

{'testlen': 295419, 'reflen': 295419, 'guess': [295419, 274042, 252665, 231288], 'correct': [295419, 274042, 252665, 231288]}
ratio: 0.9999999999999967
Bleu_1 : 99.99999999999933
Bleu_2 : 99.99999999999932
Bleu_3 : 99.9999999999993
Bleu_4 : 99.99999999999929
Rouge : 100.0
[10. 10. 10. ... 10. 10. 10.]
Cider : 1000.0


KeyboardInterrupt: ignored

In [ ]:
#tokenizer 맨앞글자 짤림..  tiny size에서 짤림.. 모델규격문제인듯
##checkpoint 확인 .. 근데 best ckpt나 기본이나 별차이는없을듯,,  encoder는 영향있을듯

#20개 토크나이저 컷
##사진 id에 따른 caption 확인
#category 활용성
#bleu 가 낮은편인데 문장 길이에 영향이 있을듯?
#PIL 로 사진읽어오는게 느린거같은데 원본코드의 방식 참조
#앞부분답이 외워지는게 학습떄 shuffle이 되고 있나?
#260248 tokens 

##SCST (not working)


In [ ]:
# model = OFAModel.from_pretrained(inference_model).to(device)
# print(inference_model)
from component.ofa.modeling_ofa import OFAModelForCaption, OFAModel
from component.ofa.tokenization_ofa import OFATokenizer #tokenizerfast로 만들고싶다..
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import torch
import os
import base64
from datetime import date, datetime
from io import BytesIO
import datetime

#inference_model = '../drive/MyDrive/NICE/model/0422_train_encoder/checkpoint-10000'  #best
#inference_model = '../drive/MyDrive/NICE/model/0424_hyperparameter/checkpoint-final'   #이미 train set에서는 만점수준..

inference_model = "../drive/MyDrive/NICE/model/cap_best/OFA-huge-caption"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = OFAModel.from_pretrained(inference_model).to(device)
tokenizer = OFATokenizer.from_pretrained('./vocab' )

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


./vocab
<super: <class 'OFATokenizer'>, <OFATokenizer object>>


In [ ]:
from component.dataset import CaptionDataset
from component.datacollator import CaptionCollator

train_caption_file = "./data/train_caption.jsonl"
train_image_file = "../drive/MyDrive/NICE/train_image.tsv"
max_seq_length = 150




train_dataset = CaptionDataset(train_caption_file, train_image_file)
data_collator = CaptionCollator(tokenizer=tokenizer, max_seq_length= max_seq_length)

2023-04-29 03:13:15.899 | INFO     | component.dataset:__init__:15 - loading data from:./data/train_caption.jsonl and ../drive/MyDrive/NICE/train_image.tsv
100%|██████████| 5000/5000 [00:00<00:00, 185841.94it/s]
2023-04-29 03:13:16.924 | INFO     | component.dataset:__init__:37 - len of data:5000


In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=4,
    collate_fn=data_collator,
)


In [ ]:
gen_target = [] #생성 토큰
gen_res = [] #디코드 문장
gt_res = [] # 정답문장
for batch in train_dataloader:

  model.eval()
  with torch.no_grad():
    output = model.generate(input_ids = batch['input_ids'], patch_images = batch['patch_images'],num_beams=5, no_repeat_ngram_size=3, max_length = 40,
                              use_cache=False  )
    gen_sentence = tokenizer.batch_decode(output, skip_special_tokens=True)
    gen_target.append(output)
    gen_res.append(gen_sentence)
    for _ in range(4):
      output = model.generate(input_ids = batch['input_ids'], patch_images = batch['patch_images'],temperature = 2.0,
                              do_sample =  True,max_length = 40,
                            #num_beams=5, no_repeat_ngram_size=3, 
                              use_cache=False )
      gen_sentence = tokenizer.batch_decode(output, skip_special_tokens=True)
      gen_target.append(output)
      gen_res.append(gen_sentence)
  caption = batch['gt']
  gt_index = batch['gt_index']
  gt_res = caption
  break

In [ ]:
dict_gt,dict_gen = {},{}
for index, id in enumerate(gt_index):
  for i in range(len(gen_res)):
    dict_gen[str(id)+'_'+ str(i%5)] = [gen_res[i][index]]
    dict_gt[str(id)+'_'+ str(i%5)] = [gt_res[index]]


In [ ]:
for i in gen_res:
  print(i)

[' edge of a bag', ' part of a sweater', ' pink shirt on a girl sitting down', ' edge of a dress']
[' two stud o piece of a case', ' legoin of a collar of blue shirt with two blue buttons', ' rubybluejimmedaughters shirt here', ' word bucketvonippiahouffle sleeved dress']
[' tip or part of suitcasebag', ' the coat has two colors on the sweater', ' small yellow tennis ball laying in the grass', ' littlettle elbow of a young girl']
['igator pouch on the leather jacket jacketbag', ' elbow buttons areaide of shrug', ' little boy head twisted upside off', ' part of an ear']
[' black leather handle of a briefcase bag leg zipper handlebreake', ' part of a knife', " small pink shirt on girl tummi's neck", ' yellow eggs piled with eggs in a basket']


In [ ]:

from pycocoevalcap.cider.cider import Cider
# gen_res_ = [{'image_id':'1', 'caption':gen_res[0][0]}]
# gt_res_ = [{'image_id':'1', 'caption':gt_res[0]}]

cider,batch_cider_scores = Cider().compute_score(dict_gt, dict_gen)

In [ ]:
CIDER_REWARD_WEIGHT =1
batch_cider_scores
scores = CIDER_REWARD_WEIGHT * batch_cider_scores

In [ ]:
batch_size = len(gt_res)
gen_res_size = len(gen_res)*batch_size
seq_per_img = gen_res_size // batch_size

sc_ = scores.reshape(batch_size, seq_per_img)
baseline = (sc_.sum(1, keepdims=True) - sc_) / (sc_.shape[1] - 1)
reward = scores.reshape(batch_size, seq_per_img)
reward = reward - baseline
reward = reward.reshape(gen_res_size)
reward = torch.as_tensor(reward, device=device, dtype=torch.float64)

tensor([-0.0296, -0.0348, -0.1435, -0.1435,  0.3513,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.1344, -0.0554, -0.0554, -0.0554,  0.0318, -0.1301,
        -0.1301,  0.1587, -0.1301,  0.2315], dtype=torch.float64)


In [ ]:
batch_size = len(gt_res)
gen_res_size = len(gen_res)*batch_size
seq_per_img = gen_res_size // batch_size

model.train()
sample_src_tokens = torch.repeat_interleave(
    batch['input_ids'], seq_per_img, dim=0
)
sample_patch_images = torch.repeat_interleave(
        batch['patch_images'], seq_per_img, dim=0
)

len_max = 0
for i in gen_target:
  if i.size(1) >len_max:
    len_max = i.size(1)
for j in range(batch_size):
  for i in range(seq_per_img ):
    padding = torch.ones(len_max - len(gen_target[i][j]), dtype = torch.int64)
    gen_padded = torch.cat((gen_target[i][j], padding))
    if i==0 and j==0:
      gen_prev_output_tokens = gen_padded.unsqueeze(0)
    else:
      gen_prev_output_tokens = torch.cat((gen_prev_output_tokens, gen_padded.unsqueeze(0)),dim=0)

net_output = model(input_ids = sample_src_tokens, patch_images = sample_patch_images, decoder_input_ids = gen_prev_output_tokens)

In [ ]:
lprobs = model.get_normalized_probs(net_output, log_probs=True) # 0~1에서 로그함수적용하므로 음수행

In [ ]:
loss = -lprobs.gather(dim=-1, index=gen_prev_output_tokens.unsqueeze(-1)).squeeze() * reward.unsqueeze(-1)

In [ ]:
pad_mask= gen_prev_output_tokens.eq(1)
loss.masked_fill_(pad_mask, 0.0)
ntokens = (~pad_mask).sum()
loss = loss.sum()

tensor(27.9917, dtype=torch.float64, grad_fn=<SumBackward0>)


##SCST huggingface (not working)


In [ ]:
from torch import nn
from transformers import Trainer
from pycocoevalcap.cider.cider import Cider

# from torch.utils.data import DataLoader

# train_dataloader = DataLoader(
#     train_dataset,
#     shuffle=True,
#     batch_size=4,
#     collate_fn=data_collator,
# )


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        ## eval sample output to get reward
        gen_target = [] #생성 토큰
        gen_res = [] #디코드 문장
        gt_res = [] # 정답문장
        for batch in train_dataloader:

          model.eval()
          with torch.no_grad():
            output = model.generate(input_ids = batch['input_ids'], patch_images = batch['patch_images'],num_beams=5, no_repeat_ngram_size=3, max_length = 40,
                                      use_cache=False  )
            gen_sentence = tokenizer.batch_decode(output, skip_special_tokens=True)
            gen_target.append(output)
            gen_res.append(gen_sentence)
            for _ in range(4):
              output = model.generate(input_ids = batch['input_ids'], patch_images = batch['patch_images'],temperature = 2.0,
                                      do_sample =  True,max_length = 40,
                                    #num_beams=5, no_repeat_ngram_size=3, 
                                      use_cache=False )
              gen_sentence = tokenizer.batch_decode(output, skip_special_tokens=True)
              gen_target.append(output)
              gen_res.append(gen_sentence)
          caption = batch['gt']
          gt_index = batch['gt_index']
          gt_res = caption
        dict_gt,dict_gen = {},{}
        for index, id in enumerate(gt_index):
          for i in range(len(gen_res)):
            dict_gen[str(id)+'_'+ str(i%5)] = [gen_res[i][index]]
            dict_gt[str(id)+'_'+ str(i%5)] = [gt_res[index]]

        cider,batch_cider_scores = Cider().compute_score(dict_gt, dict_gen)
        CIDER_REWARD_WEIGHT =1
        batch_cider_scores
        scores = CIDER_REWARD_WEIGHT * batch_cider_scores

        #scst reward
        batch_size = len(gt_res)
        gen_res_size = len(gen_res)*batch_size
        seq_per_img = gen_res_size // batch_size

        sc_ = scores.reshape(batch_size, seq_per_img)
        baseline = (sc_.sum(1, keepdims=True) - sc_) / (sc_.shape[1] - 1)
        reward = scores.reshape(batch_size, seq_per_img)
        reward = reward - baseline
        reward = reward.reshape(gen_res_size)
        reward = torch.as_tensor(reward, device=device, dtype=torch.float64)
        
        # model.train to get net output
        # batch_size = len(gt_res)
        # gen_res_size = len(gen_res)*batch_size
        # seq_per_img = gen_res_size // batch_size

        model.train()
        sample_src_tokens = torch.repeat_interleave(
            batch['input_ids'], seq_per_img, dim=0
        )
        sample_patch_images = torch.repeat_interleave(
                batch['patch_images'], seq_per_img, dim=0
        )

        len_max = 0
        for i in gen_target:
          if i.size(1) >len_max:
            len_max = i.size(1)
        for j in range(batch_size):
          for i in range(seq_per_img ):
            padding = torch.ones(len_max - len(gen_target[i][j]), dtype = torch.int64)
            gen_padded = torch.cat((gen_target[i][j], padding))
            if i==0 and j==0:
              gen_prev_output_tokens = gen_padded.unsqueeze(0)
            else:
              gen_prev_output_tokens = torch.cat((gen_prev_output_tokens, gen_padded.unsqueeze(0)),dim=0)

        net_output = model(input_ids = sample_src_tokens, patch_images = sample_patch_images, decoder_input_ids = gen_prev_output_tokens)
        # compute custom loss 
        lprobs = model.get_normalized_probs(net_output, log_probs=True) 
        loss = -lprobs.gather(dim=-1, index=gen_prev_output_tokens.unsqueeze(-1)).squeeze() * reward.unsqueeze(-1)
        pad_mask= gen_prev_output_tokens.eq(1)
        loss.masked_fill_(pad_mask, 0.0)
        ntokens = (~pad_mask).sum()
        loss = loss.sum()
        return (loss, net_output) if return_outputs else loss





In [ ]:
def compute_loss(self, model, sample, reduce=True):
    gen_target, gen_res, gt_res = self.get_generator_out(model, sample)
    reward, scores = self.get_reward_and_scores(gen_res, gt_res, device=sample["target"].device)
    net_output, gen_target_tokens = self.get_net_output(model, sample, gen_target)
    gen_lprobs, gen_target_tokens = self.get_lprobs_and_target(model, net_output, gen_target_tokens)
    loss, ntokens = scst_loss(gen_lprobs, gen_target_tokens, reward, ignore_index=self.padding_idx, reduce=reduce)
    nsentences = gen_target_tokens.size(0)

    return loss, scores.sum(), ntokens, nsentences

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python train.py --train_args_file train_args/train_ofa.json 

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
loading file ./vocab/merges.txt
loading file ./vocab/added_tokens.json
loading file ./vocab/special_tokens_map.json
loading file ./vocab/tokenizer_config.json
 86% 4291/5000 [4:40:01<44:20,  3.75s/it]./vocab
<super: <class 'OFATokenizer'>, <OFATokenizer object>>
loading file ./vocab/vocab.json
loading file ./vocab/merges.txt
loading file ./vocab/added_tokens.json
loading file ./vocab/special_tokens_map.json
loading file ./vocab/tokenizer_config.json
 86% 4292/5000 [4:40:04<44:08,  3.74s/it]./vocab
<super: <class 'OFATokenizer'>, <OFATokenizer object>>
loading file ./vocab/vocab.json
loading file ./vocab/merges.txt
loading file ./vocab/added_tokens.json
loading file ./vocab/special_tokens_map.json
loading file ./vocab/tokenizer_config.json
 86% 4293/5000 [4:40:10<49:34,  4.21s/it]./vocab
<super: <class 'OFATokenizer'>, <OFATokenizer object>>
loading file ./vocab/vocab.json
loading file ./vocab/merges.txt
loading file ./vocab/added_tokens.json
loading 

In [ ]:



def forward(self, model, sample, update_num=0, reduce=True):
    """Compute the loss for the given sample.
    Returns a tuple with three elements:
    1) the loss
    2) the sample size, which is used as the denominator for the gradient
    3) logging outputs to display while training
    """
    loss, score, ntokens, nsentences = self.compute_loss(model, sample, reduce=reduce)

    sample_size = (
        nsentences if self.sentence_avg else ntokens
    )
    logging_output = {
        "loss": loss.data,
        "score": score,
        "ntokens": ntokens,
        "nsentences": nsentences,
        "sample_size": sample_size,
    }
    return loss, sample_size, logging_output
    
def compute_loss(self, model, sample, reduce=True):
    gen_target, gen_res, gt_res = self.get_generator_out(model, sample)
    reward, scores = self.get_reward_and_scores(gen_res, gt_res, device=sample["target"].device)
    net_output, gen_target_tokens = self.get_net_output(model, sample, gen_target)
    gen_lprobs, gen_target_tokens = self.get_lprobs_and_target(model, net_output, gen_target_tokens)
    loss, ntokens = scst_loss(gen_lprobs, gen_target_tokens, reward, ignore_index=self.padding_idx, reduce=reduce)
    nsentences = gen_target_tokens.size(0)

    return loss, scores.sum(), ntokens, nsentences

def get_generator_out(self, model, sample): #1
    def decode(toks):
        hypo = toks.int().cpu()
        hypo_str = self.task.tgt_dict.string(hypo)
        hypo_str = self.task.bpe.decode(hypo_str).strip()
        return hypo, hypo_str

    model.eval()
    with torch.no_grad():
        self.task.scst_generator.model.eval()
        gen_out = self.task.scst_generator.generate([model], sample)

    gen_target = []
    gen_res = []
    gt_res = []
    for i in range(len(gen_out)):
        for j in range(len(gen_out[i])):
            hypo, hypo_str = decode(gen_out[i][j]["tokens"])
            gen_target.append(hypo)
            gen_res.append(hypo_str)
        gt_res.append(
            decode(utils.strip_pad(sample["target"][i], self.padding_idx))[1].split('&&')
        )

    return gen_target, gen_res, gt_res

def get_reward_and_scores(self, gen_res, gt_res, device): #2
    batch_size = len(gt_res)
    gen_res_size = len(gen_res)
    seq_per_img = gen_res_size // batch_size

    gt_idx = [i // seq_per_img for i in range(gen_res_size)]
    scores = self._calculate_eval_scores(gen_res, gt_idx, gt_res) ##get ciderscore
    sc_ = scores.reshape(batch_size, seq_per_img)
    baseline = (sc_.sum(1, keepdims=True) - sc_) / (sc_.shape[1] - 1)
    # sample - baseline
    reward = scores.reshape(batch_size, seq_per_img)
    reward = reward - baseline
    reward = reward.reshape(gen_res_size)
    reward = torch.as_tensor(reward, device=device, dtype=torch.float64)

    return reward, scores

def get_net_output(self, model, sample, gen_target): #3
    def merge(sample_list, eos=self.task.tgt_dict.eos(), move_eos_to_beginning=False):
        return data_utils.collate_tokens(
            sample_list,
            pad_idx=self.padding_idx,
            eos_idx=eos,
            left_pad=False,
            move_eos_to_beginning=move_eos_to_beginning,
        )

    batch_size = len(sample["target"])
    gen_target_size = len(gen_target)
    seq_per_img = gen_target_size // batch_size

    model.train()
    sample_src_tokens = torch.repeat_interleave(
        sample['net_input']['src_tokens'], seq_per_img, dim=0
    )
    sample_src_lengths = torch.repeat_interleave(
        sample['net_input']['src_lengths'], seq_per_img, dim=0
    )
    sample_patch_images = torch.repeat_interleave(
        sample['net_input']['patch_images'], seq_per_img, dim=0
    )
    sample_patch_masks = torch.repeat_interleave(
        sample['net_input']['patch_masks'], seq_per_img, dim=0
    )
    gen_prev_output_tokens = torch.as_tensor(
        merge(gen_target, eos=self.task.tgt_dict.bos(), move_eos_to_beginning=True),
        device=sample["target"].device, dtype=torch.int64
    )
    gen_target_tokens = torch.as_tensor(
        merge(gen_target), device=sample["target"].device, dtype=torch.int64
    )
    net_output = model(
        src_tokens=sample_src_tokens, src_lengths=sample_src_lengths,
        patch_images=sample_patch_images, patch_masks=sample_patch_masks,
        prev_output_tokens=gen_prev_output_tokens
    )

    return net_output, gen_target_tokens

def get_lprobs_and_target(self, model, net_output, gen_target): #4
    if self.constraint_start is not None and self.constraint_end is not None:
        net_output[0][:, :, 4:self.constraint_start] = -math.inf
        net_output[0][:, :, self.constraint_end:] = -math.inf
    lprobs = model.get_normalized_probs(net_output, log_probs=True)
    if self.ignore_prefix_size > 0:
        if getattr(lprobs, "batch_first", False):
            lprobs = lprobs[:, self.ignore_prefix_size :, :].contiguous()
            gen_target = gen_target[:, self.ignore_prefix_size :].contiguous()
        else:
            lprobs = lprobs[self.ignore_prefix_size :, :, :].contiguous()
            gen_target = gen_target[self.ignore_prefix_size :, :].contiguous()
    return lprobs, gen_target

  
def scst_loss(lprobs, target, reward, ignore_index=None, reduce=True): #5
    loss = -lprobs.gather(dim=-1, index=target.unsqueeze(-1)).squeeze() * reward.unsqueeze(-1)
    if ignore_index is not None:
        pad_mask = target.eq(ignore_index)
        loss.masked_fill_(pad_mask, 0.0)
        ntokens = (~pad_mask).sum()
    else:
        loss = loss.squeeze(-1)
        ntokens = target.numel()
    if reduce:
        loss = loss.sum()
    return loss, ntokens



  
#scst_loss 보니 reward는 scalar로 나오나? sequence 전반에 대한 sum?